## Mod 2 Assessment

In [2]:
# put all imports here
import json
import requests
import pandas as pd
import pymongo
import sqlite3
from bs4 import BeautifulSoup
from bson.json_util import loads

### SQL (10 Minutes)
There is a sqlite3 database in `assets/books.db`. The SQL to create this is also in `assets/books.sql`, if you want to run it manually (you can also import this using https://sqliteonline.com/, or run the SQL file directly).  Both have the same schema and data.

The schema has three tables. You can explore the schema in the fiddle posted above.  Please answer the following questions.

1. Connect to the database using sqlite3

In [3]:
# Your code here
# create connection & cursor
conn = sqlite3.connect("./assets/books.db")
cur = conn.cursor()

In [12]:
# function: output query result as df (default fetchall )
def grab_table():
    df = pd.DataFrame(cur.fetchall())
    df.columns = [row[0] for row in cur.description]
    return df

In [19]:
# grab tables in the database
cur.execute("""
SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';
""")
grab_table()

,name
0,author
1,book
2,book_author


2. Querying the DB: how many pages are in the book "Nine Stories"?

In [13]:
# explore book table 
cur.execute("""
SELECT * 
FROM book
LIMIT 10;
""")
grab_table()

,book_id,title,pages
0,1,The Catcher in the Rye,500
1,2,Nine Stories,600
2,3,Franny and Zooey,700
3,4,The Great Gatsby,800
4,5,Tender id the Night,200
5,6,Pride and Prejudice,30000
6,7,Professional ASP.NET 4.5 in C# and VB,3


In [21]:
# Your code here
cur.execute("""
SELECT title, pages
FROM book
WHERE title = "Nine Stories";
""")
grab_table()

,title,pages
0,Nine Stories,600


3. How many authors are from the USA?

In [22]:
# explore author table
cur.execute("""
SELECT * 
FROM author
LIMIT 10;
""")
grab_table()

,author_id,name,country
0,1,J.D. Salinger,USA
1,2,F. Scott. Fitzgerald,USA
2,3,Jane Austen,UK
3,4,Scott Hanselman,USA
4,5,Jason N. Gaylord,USA
5,6,Pranav Rastogi,India
6,7,Todd Miranda,USA
7,8,Christian Wenz,USA


In [23]:
# Your code here
cur.execute("""
SELECT count(name) as author_counts_USA
FROM author
WHERE country = 'USA';
""")
grab_table()

,author_counts_USA
0,6


4. How many authors does the book "Professional ASP.NET 4.5 in C# and VB" have?

In [26]:
# explore book_author table
cur.execute("""
SELECT * 
FROM book_author
LIMIT 5;
""")
grab_table()

,author_id,book_id
0,1,1
1,1,2
2,1,3
3,2,4
4,2,5


In [32]:
# Your code here
cur.execute("""
SELECT book.title, count(author.name) as author_count
FROM book_author
LEFT JOIN book using (book_id)
LEFT JOIN author using (author_id)
WHERE title = "Professional ASP.NET 4.5 in C# and VB";
""")
grab_table()

,title,author_count
0,Professional ASP.NET 4.5 in C# and VB,5


5. How many pages total have been written by non-American authors?

In [34]:
# Your code here
cur.execute("""
SELECT sum(pages) as non_USA_pages
FROM book_author
JOIN book using (book_id)
JOIN author using (author_id)
WHERE country != 'USA'
""")
grab_table()

,non_USA_pages
0,30003


### Object Oriented Programming (15 Minutes)
Below is the stub of some classes. Make the classes adhere to the following rules. The code beneath the cell should  run successfully.
1. Force every new instance of `WeWorkMember` to pass in a value to set the attribute `name`.
1. Give every new instance of `WeWorkMember` an attribute `caffeinated` that is set to `False`. 
1. Give `WeWorkMember` an **instance** method called `caffeinate()` that prints out "Getting coffee!" and sets the attribute `caffeinated` to `True`.
1. Make it so that `Staff` and `Student` inherit from `WeWorkMember`
1. Give `Staff` a **static** method called `cheer()` that prints out "Goooooooo Flatiron Students!"
1. Give `Students` a **class** method called `learn()` that takes in an integer and returns that number +1

In [51]:
# Modify this cell
class WeWorkMember:
    
    def __init__(self, name):
        self.name = name 
        self.caffeinated = False
    def caffeinate(self): 
        print("Getting coffee")
        self.caffeinated = True

class Staff(WeWorkMember):
    
    def __init(self, name):
        super().__init__(None)
        self.name = name
    
    @staticmethod
    def cheer():
        print("Goooooooo Flatiron Students!")
        
class Student(WeWorkMember):
    
    def __init(self):
        super().__init__(name)
    
    @classmethod
    def learn(cls, num):
        if isinstance(num, int):
            print(num+1)
        else:
            raise ValueError ("expected input an int type value")

In [52]:
# DO NOT MODIFY this cell. if you want to add more tests, add them in a new cell!
try:
    nobody = Staff()
    raise RuntimeError('Staff needs a name!')
except TypeError:
    pass

andy = Staff('Andy')
assert andy.caffeinated is False
andy.caffeinate()
assert andy.caffeinated is True

Staff.cheer()

bobby = Student('Bobby Tables')
assert bobby.caffeinated is False
bobby.caffeinate()
assert bobby.caffeinated is True

Student.learn(99)

Getting coffee
Goooooooo Flatiron Students!
Getting coffee
100


### APIs (10 Minutes)
Using this API (http://www.nokeynoshade.party/api/judges) about RuPaul's Drag Race (docs: https://drag-race-api.readme.io/docs/get-all-judges), tell me how many judges of each **`type`** there were in the first 50 records the API returns.
 - the 50 judges will have **`id`** ranging from 1 to 53
 - you can do the aggregation in pure Python, Pandas, SQL-- whatever's easiest for you

In [94]:
response = requests.get("http://www.nokeynoshade.party/api/judges")

In [95]:
response.headers

{'Server': 'Cowboy', 'Connection': 'keep-alive', 'X-Powered-By': 'Express', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, PATCH, OPTIONS', 'Access-Control-Allow-Headers': 'Origin, X-Requested-With, Content-Type, Accept', 'X-Ratelimit-Limit': '1500', 'X-Ratelimit-Remaining': '1434', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '5257', 'Etag': 'W/"1489-SdEZzZSb95F7HnHTsWoPM8cgbC4"', 'Date': 'Wed, 03 Jul 2019 14:43:04 GMT', 'Via': '1.1 vegur'}

In [96]:
judges = response.json()

In [98]:
type(judges)

list

In [103]:
for i in range(25, 51):
    url = "http://www.nokeynoshade.party/api/judges/" + str(i)
    new = requests.get(url).json()
    judges.append(new)

In [104]:
judges

[{'id': 1,
  'name': 'Rupaul',
  'image_url': 'https://vignette.wikia.nocookie.net/logosrupaulsdragrace/images/b/ba/Rupaul_blackpink_final.jpg/revision/latest/scale-to-width-down/350?cb=20110731183922',
  'bio': 'Drag performer, actor, television host, and recording artist',
  'type': 'regular',
  'createdAt': '2018-01-11T00:43:31.258Z',
  'updatedAt': '2018-01-11T00:43:31.258Z'},
 {'id': 2,
  'name': 'Bob Mackie',
  'image_url': None,
  'bio': 'Fashion Designer',
  'type': 'guest',
  'createdAt': '2017-08-22T00:27:40.731Z',
  'updatedAt': '2017-08-22T00:27:40.731Z'},
 {'id': 3,
  'name': 'Mike Ruiz',
  'image_url': 'https://vignette3.wikia.nocookie.net/logosrupaulsdragrace/images/7/7c/Mike_Ruiz.jpg/revision/latest/scale-to-width-down/350?cb=20110731184513',
  'bio': 'Photographer',
  'type': 'regular',
  'createdAt': '2017-08-22T00:46:35.279Z',
  'updatedAt': '2017-08-22T00:46:35.279Z'},
 {'id': 4,
  'name': 'Michelle Williams',
  'image_url': 'https://upload.wikimedia.org/wikipedia/c

In [109]:
judges[0]['type']

'regular'

In [118]:
## id = 43-45 ('internal server error')
judge_types = []
for judge in judges:
    if 'type' in judge.keys():
        judge_types.append(judge['type'])
    else: 
        continue

In [121]:
# count types of judge
print(f"{len(set(judge_types))} types of judge")

3 types of judge


### Web Scraping (15 Minutes)
Scrape the website [http://quotes.toscrape.com/](http://quotes.toscrape.com/).  Turn each quote on the page into a Pandas dataframe consisting of two columns: the author of the quote and the quote itself. The final result should be a dataframe of 10 rows (not counting header) and 2 columns (not counting index).

### NoSQL (10 Minutes)
Answer all of the following questions by querying Mongo and manipulating the results in Python
1. Load data from `assets/grades.json` into Mongo (this code is written for you)

In [ ]:
# you shouldn't need to edit this cell!
with open('assets/grades.jsonl') as f:
    # loads() comes from the bson library
    file_data = [loads(line) for line in f.readlines()]

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["mod2db"]
coll = db["testcoll"]

coll.insert_many(file_data)

2. How many records are there total?

In [ ]:
# Your code here


3. How many students have taken the class with `class_id` = **29**?

In [ ]:
# Your code here


4. Super bonus question: for student **12** in class **23**, what grade did they get on their exam?

In [ ]:
# Your code here


In [ ]:
# for when you're done with this portion, this deletes the data we added.
client.drop_database("mod2db")

## Assessment submission (2 Minutes)
Please save your completed file as `mod2-assessment.ipynb` and upload it using [this form](https://docs.google.com/forms/d/e/1FAIpQLSczNVM9Q6Dr3w1YR3kkg4PNaOsfDO-qIvh1wLy6rR0SEwTwIQ/viewform?usp=sf_link)